In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [3]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [4]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

In [5]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, delta_up[1:W,1:hours])
@variable(model, delta_down[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W*(
        scenarios[i][hour,"price"] * p_DA[hour]
     + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
      - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
      ) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)



# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

Optimal solution found
Objective value: 130697.43588726835


In [6]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))
println("delta_up: ", value.(delta_up))
println("delta_down: ", value.(delta_down))

Objective value: 130697.43588726835
p_DA: [64.39741935483872, 14.389693451612905, 56.90564516129032, 50.954193548387096, 43.07419354838709, 45.95790322580645, 145.9269354516129, 13.34106441935484, 1.1038967741935486, 146.94499998387096, 21.6095805, 0.7419306451612904, 142.91516125806453, 50.27709674193549, 135.9887096612903, 53.12935479032258, 146.0248386935484, 149.60145158064518, 16.439596645161288, 152.74596772580645, 4.493516129032258, 49.94887096774193, 6.808774177419355, 149.57870966129033]
delta: [30.647741919354843 80.69353230645162 45.33887095161292 52.92129020967742 57.01564514516131 54.276290258064506 -47.34850807258064 88.63345169354838 105.82868385483872 -35.65629037096774 87.20945166129033 103.86161764516127 -41.94677422580645 47.68532249999999 -47.478225822580626 25.548709677419353 -82.71500004838711 -104.93290327419356 15.714419403225811 -122.93154845161291 26.57029022580645 -22.718967790322573 16.735032145161288 -125.07222585483872; -14.778387096774203 31.8080484677419

In [7]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])
#, delta = Float64[], delta_up = Float64[], delta_down = Float64[]

#make a table with the results
scenario = 1
for hour in 1:hours
    push!(result_df, [hour, value(p_DA[hour])])
    #, value(delta[scenario,hour]), value(delta_up[scenario,hour]), value(delta_down[scenario,hour])
end

pretty_table(result_df, backend = Val(:latex))

In [8]:
profit_df = DataFrame(scenario = Int[], profit = Float64[])

for i in 1:W
    profit = 0
    for hour in 1:hours
        profit += (
            scenarios[i][hour,"price"] * value(p_DA[hour])
        + value(delta_up[i,hour]) * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - value(delta_down[i,hour]) * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
        )
    end
    push!(profit_df, [i, profit])
end

#return average of profit
println("Average profit: ", sum(profit_df[!,"profit"])/W)

\begin{tabular}{rr}
  \hline
  \textbf{hour} & \textbf{p\_DA} \\
  \texttt{Int64} & \texttt{Float64} \\\hline
  1 & 64.3974 \\
  2 & 14.3897 \\
  3 & 56.9056 \\
  4 & 50.9542 \\
  5 & 43.0742 \\
  6 & 45.9579 \\
  7 & 145.927 \\
  8 & 13.3411 \\
  9 & 1.1039 \\
  10 & 146.945 \\
  11 & 21.6096 \\
  12 & 0.741931 \\
  13 & 142.915 \\
  14 & 50.2771 \\
  15 & 135.989 \\
  16 & 53.1294 \\
  17 & 146.025 \\
  18 & 149.601 \\
  19 & 16.4396 \\
  20 & 152.746 \\
  21 & 4.49352 \\
  22 & 49.9489 \\
  23 & 6.80877 \\
  24 & 149.579 \\\hline
\end{tabular}
Average profit: 130697.43588726828


In [9]:
CSV.write("1_2_results/profits_1_2.csv", profit_df)
CSV.write("1_2_results/market_results_1_2.csv", result_df)

"1_2_results/market_results_1_2.csv"